# _Automate Documentation, Comments, and Unit Tests for Python Code_  
# _Convert Python to Optimized C++ Code_

## Overview  
It is a Gradio-powered tool designed to automate essential but time-consuming Python development tasks. It streamlines documentation, unit testing, and Python-to-C++ code conversion with AI-driven assistance.  

### Key Features  
**Auto-Generate Docstrings & Comments** – Instantly improve code clarity and maintainability.  
**Unit Test Generation** – Ensure reliability with AI-generated test cases.  
**Python to C++ Conversion** – Seamlessly translate Python code to C++ with execution support.  

### Model used
1. ***Frontier model***- gpt-4o-mini
2. ***Open source model*** - meta-llama/Meta-Llama-3-8B-Instruct

In [1]:
!pip install -q openai gradio huggingface_hub transformers bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/5

In [2]:
# Imports
from google.colab import userdata
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TextStreamer
from openai import OpenAI
import gradio as gr
import torch

In [3]:
hf_token = userdata.get('HF_TOKEN')
login(hf_token)

In [4]:
gpt = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [5]:
# Define constants

OPENAI_MODEL = "gpt-4o-mini"
LLAMA = "meta-llama/Meta-Llama-3-8B-Instruct"

MODEL_LIST = ["GPT", "LLAMA"]

TASK_COMMENT = "commenting"
TASK_TEST = "testing"
TASK_CONVERT = "converting"

PYTHON_SCRIPT_DEFAULT = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [6]:
#### System prompts

# System prompts for commenting
SYSTEM_PROMPT_COMMENTS = """
You are an AI model specializing in enhancing Python code documentation.
Generate detailed and precise docstrings and inline comments for the provided Python code.
Ensure the docstrings clearly describe the purpose, parameters, and return values of each function.
Inline comments should explain complex or non-obvious code segments.
Do not include any introductions, explanations, conclusions, or additional context.
Return only the updated Python code enclosed within ```python ... ``` for proper formatting and syntax highlighting.
"""

# System prompts for testing
SYSTEM_PROMPT_TESTS = """
You are an AI model specializing in generating comprehensive unit tests for Python code.
Create Python unit tests that thoroughly validate the functionality of the given code.
Use the `unittest` framework and ensure edge cases and error conditions are tested.
Do not include any comments, introductions, explanations, conclusions, or additional context.
Return only the unit test code enclosed within ```python ... ``` for proper formatting and syntax highlighting.
"""

# System prompts for code cenversion
SYSTEM_PROMPT_CONVERT = """
You are an AI model specializing in high-performance code translation.
Translate the given Python code into equivalent, optimized C++ code.
Focus on:
- Using efficient data structures and algorithms.
- Avoiding unnecessary memory allocations and computational overhead.
- Ensuring minimal risk of integer overflow by using appropriate data types.
- Leveraging the C++ Standard Library (e.g., `<vector>`, `<algorithm>`) for performance and readability.
Produce concise and efficient C++ code that matches the functionality of the original Python code.
Do not include any comments, introductions, explanations, conclusions, or additional context..
Return only the C++ code enclosed within ```cpp ... ``` for proper formatting and syntax highlighting.
"""

In [7]:
### User prompts

# User prompts for commenting
def user_prompt_comments(python_code):
  user_prompt = f"""
Add detailed docstrings and inline comments to the following Python code:
```python
{python_code}
```
"""
  return user_prompt

# User prompts for testing
def user_prompt_tests(python_code):
    user_prompt = f"""
Generate unit tests for the following Python code using the `unittest` framework:

```python
{python_code}
```
"""
    return user_prompt

# User prompts for code converting
def user_prompt_convert(python_code):
    user_prompt = f"""
Convert the following Python code into C++:

```python
{python_code}
```
"""
    return user_prompt

In [8]:
# Function for calling OpenAI model

def stream_gpt(system_prompt, user_prompt):
  print("Inside GPT")
  stream = gpt.chat.completions.create(
      model=OPENAI_MODEL,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": user_prompt}
      ],
      stream=True
  )

  reply = ""
  for chunk in stream:
    reply += chunk.choices[0].delta.content or ""
    yield reply.replace("```python\n", "").replace("```cpp\n", "").replace("```", "")

In [9]:
# Quantization Config - this allows us to load the model into memory and use less memory

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [59]:
# Load LLAMA model

model_llama = AutoModelForCausalLM.from_pretrained(
    LLAMA,
    torch_dtype="auto",
    device_map="auto",
    quantization_config=quant_config
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [60]:
# Function for calling open source LLAMA model

def stream_llama(system_prompt, user_prompt):
  print("Inside LLAMA")
  messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
  ]

  tokenizer = AutoTokenizer.from_pretrained(LLAMA)
  tokenizer.pad_token = tokenizer.eos_token
  inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

  reply = ""
  for token in model_llama.generate(inputs, max_new_tokens=3000):
      decoded_text = tokenizer.decode(token, skip_special_tokens=True)
      reply += decoded_text
      yield reply.replace("```python\n", "").replace("```cpp\n", "").replace("```", "")

In [61]:
# Function for setting prompts for user specific tasks

def set_prompts(user_input, task):

  task = task.lower()
  if task == TASK_COMMENT.lower():
    system_prompt = SYSTEM_PROMPT_COMMENTS
    user_prompt = user_prompt_comments(user_input)
  elif task == TASK_TEST.lower():
    system_prompt = SYSTEM_PROMPT_TESTS
    user_prompt = user_prompt_tests(user_input)
  elif task == TASK_CONVERT.lower():
    system_prompt = SYSTEM_PROMPT_CONVERT
    user_prompt = user_prompt_convert(user_input)
  else:
    return None, None

  return system_prompt, user_prompt

In [62]:
# Function for streaming response depending on the selected model

def stream_respone(user_input, task, model):

  system_prompt, user_prompt = set_prompts(user_input, task)
  if system_prompt == None or user_prompt == None:
    raise ValueError("Invalid task")

  if model == "GPT":
    yield from stream_gpt(system_prompt, user_prompt)
  elif model == "LLAMA":
    yield from stream_llama(system_prompt, user_prompt)
  else:
    raise ValueError("Invalid model")

In [65]:
# Function for generating comments
def generate_comments(python_code, selected_model):

  yield from stream_respone(python_code, TASK_COMMENT, selected_model)
  return

# Function for testing
def generate_tests(python_code, selected_model):

  yield from stream_respone(python_code, TASK_TEST, selected_model)
  return

# Function for code conversion
def convert_code(python_code, selected_model):

  yield from stream_respone(python_code, TASK_CONVERT, selected_model)
  return

In [69]:
css = """
.python {
    background-color: #377ef0;
    color: #ffffff;
    padding: 0.5em;
    border-radius: 5px; /* Slightly rounded corners */
}
.response {
    background-color: #00549e;
    color: #ffffff;
    padding: 0.5em;
    border-radius: 5px;
"""

In [73]:
# Create UI with Gradio

with gr.Blocks() as ui:
  with gr.Row():
    python_code = gr.Textbox(label="Python code:", value=PYTHON_SCRIPT_DEFAULT, lines=10, elem_classes=["python"])
    response = gr.Textbox(label="LLM Response:", lines=10, elem_classes=["response"])

  with gr.Row():
    model = gr.Dropdown(["GPT", "LLAMA"], label="Select model", value="GPT")
  with gr.Row():
    comment_button = gr.Button("Comment Code")
    tests_button = gr.Button("Create Test Cases")
    convert_button = gr.Button("Convert to C++")

  comment_button.click(generate_comments, inputs=[python_code, model], outputs=[response])
  tests_button.click(generate_tests, inputs=[python_code, model], outputs=[response])
  convert_button.click(convert_code, inputs=[python_code, model], outputs=[response])

  ui.launch(inbrowser=True, share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3be07d42d2a4db6c9e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://3be07d42d2a4db6c9e.gradio.live


In [ ]:
# Clean up

del model_llama
torch.cuda.empty_cache()